In [6]:
from redcap import Project
import pandas as pd

api_url = 'https://ruff.mrc.gm:8443/redcap/api/'
api_key = 'E5FEBF04C14581CF182ABE8107AB8B50'
project = Project(api_url, api_key)

## COMPLETED RECRUITMENT AND NOT YET WEEK 20

In [7]:
import datetime

# Specify the fields you want to extract
con_uss = project.export_records(forms=['consent','ultrasound_scan'])
con_uss=pd.DataFrame(con_uss)

# Specify the fields you want to extract
df_data_uss=con_uss[['con_participantid_q1', 'con_participant_eden_num_q11', 'ultsc_date', 'ga_weeks','ga_days','con_cluster_q12','ultrascan_exp_date','redcap_event_name']]
df_data_uss=pd.DataFrame(df_data_uss)
df_data_uss = df_data_uss[
    (df_data_uss['redcap_event_name'] == 'recruitment_arm_1') &
    (df_data_uss['ultsc_date'].notnull()) &
    (pd.to_numeric(df_data_uss['ga_weeks']) <= 20)
]

## EXCLUDE ALL WITHDRAWALS FROM THE DATA##
#Read from the ones that complete withdrawals: "Withdrawals.csv"
df_withdrawals = pd.read_csv('Withdrawals.csv')

# Merge the dataframes on 'participant_id'
con_merged_df = pd.merge(df_data_uss, df_withdrawals, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
df_data_uss= df_data_uss[~df_data_uss['con_participantid_q1'].isin(con_merged_df['con_participantid_q1'])]


##AD-HOC DATA FRAME##
# Convert 'ga_weeks' and 'ga_days' columns to numeric type
df_data_uss['ga_weeks'] = pd.to_numeric(df_data_uss['ga_weeks'], errors='coerce')
df_data_uss['ga_days'] = pd.to_numeric(df_data_uss['ga_days'], errors='coerce')

# Calculate today's date
today = pd.Timestamp.today()

# Extract the required columns from the DataFrame
participant_id = df_data_uss['con_participantid_q1']
uss_date = pd.to_datetime(df_data_uss['ultsc_date'])
Ga_weeks = df_data_uss['ga_weeks']
Ga_days = df_data_uss['ga_days']
eden_num = df_data_uss['con_participant_eden_num_q11']


# Calculate the total gestational age in days
total_ga_days = (Ga_weeks * 7) + Ga_days

# Calculate the estimated due date (EDD)
edd = uss_date + pd.to_timedelta(280 - total_ga_days, unit='days')

# Calculate the current gestational age
days_diff = (today - uss_date).dt.days
current_ga = (days_diff + total_ga_days) / 7

## This data frame is mainted to put together the data and maintain the variable name 'con_participantid_q1' for easy exclusion of participants who completed the wk20.


enrol_df = pd.DataFrame({
    'con_participantid_q1': participant_id,
    'Cluster':df_data_uss['con_cluster_q12'],
    'USS Date': uss_date,
    'EDEN | Infant Number': df_data_uss['con_participant_eden_num_q11'],
    'Gestational Age (Weeks)': Ga_weeks,
    'Gestational Age (Days)': Ga_days,
    '(Expected)Delivery Date ((E)DD)':edd,
    'Current Gestational Age': current_ga,
    'Visit': 'enrolled'
})




## ALL DATA THAT ARE IN THE WEEK 20.CSV EXCLUDED
#Read from the ones that complete week 20: "Week 20.csv"
wk20_uss = pd.read_csv('Week 20.csv')

# Merge the dataframes on 'participant_id'
wk20_merged_df = pd.merge(df_data_uss, wk20_uss, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
enrol_df = enrol_df[~enrol_df['con_participantid_q1'].isin(wk20_merged_df['con_participantid_q1'])]

# Get today's date and time


In [8]:
##File naming convention
# Get today's date and time
now = datetime.datetime.now()

# Format the current date and time
current_datetime = now.strftime('%d%m%Y_%H%M%S')

# Create the filename with the current date and time
filename = f'Enrolled_participants_{current_datetime}.csv'

 #Save the filtered DataFrame to a CSV file
enrol_df.to_csv(filename, index=False)